In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import print_function
import os,sys
sys.path.append('../')

## Math and dataFrame
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

import matplotlib.pyplot as plt
import xgboost
#ML
from sklearn.model_selection import cross_val_predict, StratifiedKFold, KFold
from sklearn.metrics import log_loss, accuracy_score, f1_score, confusion_matrix, roc_auc_score

/Users/saizhang/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
label_cols = ['toxic', 'severe_toxic', 'obscene',  'threat', 'insult', 'identity_hate']

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['dirtyness'] = train.apply(lambda x: x.iloc[2::].sum(), axis = 1)
test['dirtyness'] = test.apply(lambda x: x.iloc[2::].sum(), axis = 1)

COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

print("train set len ", len(train) )
print("test set len ", len(test) )
print("clean samples", len(train[train['dirtyness'] == 0]))
print("toxic samples", len(train[train['dirtyness'] != 0]))

x=train.iloc[:,2:].sum()
#plot
plt.figure(figsize=(8,4))
ax= sns.barplot(x.index, x.values, alpha=0.8)

display(train[:10])

train set len  159571
test set len  153164
clean samples 143346
toxic samples 16225


id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   
5  00025465d4725e87  "\n\nCongratulations from me as well, use the ...      0   
6  0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1   
7  00031b1e95af7921  Your vandalism to the Matt Shirvington article...      0   
8  00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...      0   
9  00040093b2687caa  alignment on this subject and which are contra...      0   

   severe_toxic  obscene  threat  insult  identity_hate  dirtyness  
0             0        0       0       0              0          0  
1             0        0       0       0              0          0  
2             0        0       0       0              0          0  
3             0        0       0       0              0          0  
4             0        0       0       0              0          0  
5             0        0       0       0              0          0  
6             1        1       0       1              0          4  
7             0        0       0       0              0          0  
8             0        0       0       0              0          0  
9             0        0       0       0              0          0

In [4]:
print (pd.concat([train, test]).iloc[159571+1])
print (train.shape)
print (test.shape)

comment_text     == From RfC == \n\n The title is fine as it is...
dirtyness                                                        0
id                                                0000247867823ef7
identity_hate                                                  NaN
insult                                                         NaN
obscene                                                        NaN
severe_toxic                                                   NaN
threat                                                         NaN
toxic                                                          NaN
Name: 1, dtype: object
(159571, 9)
(153164, 3)


In [21]:
#global config
FEstrategy = ['tfidf', 'tfIdf_charNGram']
modelName = 'xgb'
runNumber = 1
reSample = False
scaling = 'noScale'

FEstrategy_str = '+'.join(FEstrategy)
modelNameKey = '_'.join([modelName, FEstrategy_str, str(reSample), scaling, str(runNumber)])
print("modelNameKey - {}".format(modelNameKey))

modelNameKey - xgb_tfidf+tfIdf_charNGram_False_noScale_1


In [6]:
# tokenize via tfIdf
from models.FeatureExtraction import FeatureExtraction

def featureEx(FEstrategy):
    fe = FeatureExtraction()
    print("feature extraction with strategy {}".format(FEstrategy))
    if FEstrategy == 'tfidf':
        term_doc = fe.tfIdf(pd.concat([train, test]), 'comment_text')
        
    elif FEstrategy == 'tfIdf_charNGram':
        term_doc = fe.tfIdf_charNGram(pd.concat([train, test]), 'comment_text')
    
    elif FEstrategy == 'keywordTermFreq':
        keyword_dir = '../../ZhiHaoSun/'
        keyfiles = [
                keyword_dir + 'toxic_words.txt',
                keyword_dir + 'identity_hate_words.txt',
                keyword_dir + 'insult_words.txt',
                keyword_dir + 'obscene_words.txt',
                keyword_dir + 'threat_words.txt',
                keyword_dir + 'identity_hate_words.txt',
                ]


        term_doc = fe.tfKeyWordEnsemble(
                pd.concat([train, test]), n_feature = 80000, vocabfile = keyfiles,
                COMMENT = 'comment_text'
                )
        assert(term_doc.shape[0] == len(train) + len(test))
        
    elif FEstrategy == 'charNGram':
        term_doc = fe.charNGram(pd.concat([train, test]), n_feature = 15000, COMMENT = 'comment_text')
        
    else:
        raise ValueError('Undefined fe strategy')
    
    return term_doc

term_doc_list = []
for i in FEstrategy:
    print("FE - {}".format(i))
    term_doc_list.append( featureEx(i) )


FE - tfidf
feature extraction with strategy tfidf


/Users/saizhang/anaconda2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


FE - tfIdf_charNGram
feature extraction with strategy tfIdf_charNGram


In [7]:
#concatenate

# for i in term_doc_list:
#     print(type(i).__name__)
#     print(i.shape)
    
term_doc = scipy.sparse.hstack(term_doc_list)

In [8]:
def scaleFeature(scalingStrategy, term_doc):
    
    if scalingStrategy == 'stdScale':
        from sklearn.preprocessing import scale
        term_doc = scale(term_doc, with_mean=False, with_std=True, copy=True)
    elif scalingStrategy == 'noScale':
        term_doc = term_doc;
    elif scalingStrategy == 'maxAbsScale':
        from sklearn.preprocessing import MaxAbsScaler
        sc = MaxAbsScaler( copy=True)
        term_doc = sc.fit_transform(term_doc)
        
    else:
        raise ValueError("undefined scaling strategy")
        
    return term_doc

term_doc = scaleFeature(scaling, term_doc)

In [9]:
# minterm = np.min(term_doc, 0)
# maxterm = np.max(term_doc, 1)

In [10]:
# plt.hist(maxterm.toarray().squeeze())
# plt.show(block = False)

In [11]:
trn_term_doc = term_doc.tocsr()[0:len(train), :]
test_term_doc = term_doc.tocsr()[len(train)::, :]
print ("feature shape = {}".format(trn_term_doc.shape))
assert( trn_term_doc.shape[0] == train[label_cols].values.shape[0] )

feature shape = (159571, 50000)


In [27]:
# model factory 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

def buildModel(modelName):
    print("Building model with name {}".format(modelName))
    if modelName == 'rf':
        from sklearn.ensemble import RandomForestClassifier
        # Random Forest
        #https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/
        clf =  RandomForestClassifier(
                    max_depth= 5,
                    n_estimators = 10,
                    min_samples_leaf = 20,
                    max_features = None,
                    class_weight = 'balanced'
                    #class_weight = {0:0.9, 1:0.1}
                    #class_weight = {0:1, 1:100}
                    );

        model = clf
    
    elif modelName == 'ada':

        # Adaboost DT

        # assert(np.array_equal(y.unique(),  np.array([0,1])) )
        # lenLabel0 = np.sum(y == 0)
        # lenLabel1 = np.sum(y == 1)
        # print('label0 - {}'.format(lenLabel0) )
        # print('label1 - {}'.format(lenLabel1) )
        # #len( trn_term_doc[y == 0] )
        # #len( trn_term_doc[y == 1] )
        # #trn_term_doc[]
        # sampleO = trn_term_doc.tocsr()[np.where(y == 0)[0], : ]
        # sample1 = trn_term_doc.tocsr()[np.where(y == 1)[0], : ]
        # assert(sampleO.shape[0] == lenLabel0)
        # assert(sample1.shape[0] == lenLabel1)

        # if lenLabel1 < lenLabel0:
        #     print("Resample label 1")
        #     sample1 = resample(sample1, 
        #         replace=True,     # sample with replacement
        #         n_samples=lenLabel0,    # to match majority class
        #         random_state=123) # reproducible results
        # elif lenLabel0 < lenLabel1:
        #     print("Resample label 0")
        #     sample0 = resample(sample0, 
        #         replace=True,     # sample with replacement
        #         n_samples=lenLabel1,    # to match majority class
        #         random_state=123) # reproducible results

        # reSample = scipy.sparse.vstack([sampleO, sample1])
        # assert(sampleO.shape[0] == sample1.shape[0])
        # reLabel = np.hstack( [ np.zeros(sampleO.shape[0]), np.ones(sample1.shape[0]) ] )
        # assert( reSample.shape[0] == len(reLabel) )
        # if doShuffle == True:
        #     reSample, reLabel = shuffle(reSample, reLabel, random_state=0)

        from sklearn.ensemble import RandomForestClassifier
        from sklearn.ensemble import AdaBoostClassifier
        from sklearn.tree import DecisionTreeClassifier
        
        model = {}
        for lc in label_cols:
            model[lc] = AdaBoostClassifier(
            DecisionTreeClassifier(max_depth=2),
            n_estimators=30,
            learning_rate=1)


    elif modelName == 'SVC':

        model = {}
        for lc in label_cols:
            model[lc] = LogisticRegression( 
                solver = 'sag'
              #penalty="l2", dual=True, 
              #tol=0.0001, C=1, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=32, 
              # max_iter=100, verbose=0, 
              # warm_start=False, n_jobs=1
            )

#             svm_dict[lc] = SVC(C=1.0, kernel="poly", degree=2, gamma="auto", 
#                                coef0=0.0, shrinking=True, probability=True, 
#                                tol=0.001, cache_size=200, class_weight=None, 
#                                verbose=False, max_iter=-1, random_state=32)
    
    elif modelName == 'xgb':
        from xgboost import XGBClassifier        
        model = {}
        for lc in label_cols:
            model[lc] =  XGBClassifier (eta=0.3,min_child_weight=6,
                                        depth=100,num_round=10,threads=8)
#             XGBClassifier(max_depth=3, learning_rate=0.1, 
#                           n_estimators=100, silent=True, 
#                           objective='binary:logistic', 
#                           booster='gbtree', n_jobs=1, 
#                           nthread=None, gamma=0, min_child_weight=1,
#                           max_delta_step=0, subsample=1, 
#                           colsample_bytree=1, colsample_bylevel=1, reg_alpha=0,
#                           reg_lambda=1, scale_pos_weight=1, base_score=0.5, 
#                           random_state=0, seed=None, missing=None, **kwargs)
    else:
        raise ValueError('undefined model name')
    
    return model


model = buildModel(modelName)
print ("build model with type {}".format(type(model).__name__))

Building model with name xgb
build model with type dict


In [28]:
# resample
def reSampleInput(reSample, trn_term_doc, label, train, test):
    trn_weights = None
    fe = FeatureExtraction()

    if reSample == True:
        trn_term_doc_re, label_re = fe.reSample(trn_term_doc , y = train[lc])

    elif reSample == False:
        trn_term_doc_re, label_re = trn_term_doc , train[lc]

    elif reSample == 'covCorrection':
        trn_term_doc_re, label_re = trn_term_doc , train[lc]
        
        term_doc = fe.tfIdf(pd.concat([train, test]), 'comment_text')
        trn_term_doc = term_doc.tocsr()[0:len(train), :]
        test_term_doc = term_doc.tocsr()[len(train)::, :]

        trn_weights = fe.covarianceShiftCorrection(trn_term_doc, test_term_doc)
    else:
        raise ValueError("resample strategy not defined")
    return trn_term_doc_re, label_re, trn_weights


In [29]:
#cv context
def evaluate_features(X, y, clf=None, trn_weights = None):
    """General helper function for evaluating effectiveness of passed features in ML model
    
    Prints out Log loss, accuracy, and confusion matrix with 3-fold stratified cross-validation
    
    Args:
        X (array-like): Features array. Shape (n_samples, n_features)
        
        y (array-like): Labels array. Shape (n_samples,)
        
        clf: Classifier to use. If None, default Log reg is use.
    """
    if clf is None:
        raise ValueError("clf NOne")
    
    probas = cross_val_predict(clf, X, y, cv=KFold(random_state=8), 
                              n_jobs=-1, method='predict_proba', verbose=2,
                              fit_params = {'sample_weight': trn_weights}
                              )
    pred_indices = np.argmax(probas, axis=1)
    classes = np.unique(y)
    preds = classes[pred_indices]
    print('Log loss: {}'.format(log_loss(y, probas)))
    print('Accuracy: {}'.format(accuracy_score(y, preds)))
    print('F1 score: {}'.format(f1_score(y, preds)))
    print('Auc score: {}'.format(roc_auc_score(y, preds)))
    print( confusion_matrix(y, preds) )
    


In [ ]:
#fit 
if type(model).__name__ == 'dict':
    for lc in label_cols:
        print("Trainig for {}".format(lc) )
        trn_term_doc_re, label_re, trn_weights = reSampleInput(reSample, 
                                                               trn_term_doc, train[lc], train, test)
        print(trn_weights)
        
        #evaluate_features(trn_term_doc_re, label_re, model[lc], trn_weights = trn_weights)
        
        model[lc].fit(trn_term_doc_re, label_re)
else:
    
    trn_term_doc_re, label_re, trn_weights = reSampleInput(reSample, 
                                                               trn_term_doc, train[lc], train, test)
    print(trn_weights)
        
    #evaluate_features(trn_term_doc_re, label_re, model, trn_weights = trn_weights)
        
    model.fit(trn_term_doc_re, label_re)


In [16]:
from models.TrainedModel import TrainedModel
import modelDB

class TrainedModelSVC(TrainedModel):
    def __init__(self, md = None):
        super(TrainedModelSVC, self).__init__(md)
    
    def predict(self, test, **kwargs):
        print ("Predict using Model: ")
        for i in self.md.keys():  
            print( "{} - {}".format(i, type(self.md[i]).__name__ ) )
            
        res = {}
        for topic, submd in zip( self.md.keys(), self.md.values() ):
            print( " predicitng topic {}".format(topic))
            res[topic] = zip(*submd.predict_proba(test))[1] #obtain proba
        
        #important to keep the order as required by the submission file
        testid = kwargs['testid'] 
        #print (testid)
        dfres = pd.DataFrame(res)
        dfres['id'] = testid
        
        #print(dfres.shape)
        assert(dfres.shape[0] == test.shape[0])
        
        #reshape to submission file format
        dfres = dfres[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]
        
        return dfres
    
    def _save(self, mDB, nameKey, modelpath, **kwargs):
        '''
        save model into model data base
        :mDB: meta data frame storing all models info
        :nameKey: unique identifier for each saved model
        :modelpath: subdir inside modelDB dir, e.g. if modelDB is /root/modelDB, then modelpath is /cnn
        '''
        print("saving model")
        import joblib
        import datetime
        mdnameAbs = os.path.join(modelDB.MODEL_DB_ROOT, modelpath, nameKey + '.sav' )
        mdname = os.path.join(modelpath, nameKey + '.sav' )


        joblib.dump(self.md, mdnameAbs)
        #db schema
        # 'modelName', type {rnn, cnn, rf}, date, model
        print( "Info: custom saving options" )
        for i in kwargs:
            print("{} - {}".format(i , kwargs[i]))
            
        newRow = pd.DataFrame({
            'modelName': [nameKey],
            'type': kwargs['modelType'],
            'subType': 'TfIdf',
            'date': datetime.datetime.now().strftime("%Y-%m-%d"),
            'model' : mdname
        }
        )
        
        #add a new row 
        mDB = mDB.append(newRow, ignore_index = True)

        display(mDB)

        return mDB
    
    def load(self, mDB, nameKey, modelpath):
        '''
        :mDB: meta data frame storing all models info
        :nameKey: unique identifier for each saved model
        :modelpath: subdir inside modelDB dir, e.g. if modelDB is /root/modelDB, then modelpath is /cnn
        '''
        if (mDB.empty) or mDB[ mDB['modelName'] == nameKey ].empty:
            raise VaueError("Model name does not exist")
        print("loadModel")
        import joblib
        assert(len(mDB[mDB['modelName'] == nameKey]) == 1)
        mdfile = mDB[mDB['modelName'] == nameKey].iloc[0]['model']
        
        mdnameAbs = os.path.join(modelDB.MODEL_DB_ROOT, mdfile )
        
        loaded_model = joblib.load(mdnameAbs)
        assert(self.md is None)
        self.setModel(loaded_model)
        
        return self


mySVC = TrainedModelSVC(model)
dfres = mySVC.predict(test_term_doc, testid = test['id'])

Predict using Model: 
severe_toxic - LogisticRegression
identity_hate - LogisticRegression
obscene - LogisticRegression
insult - LogisticRegression
threat - LogisticRegression
toxic - LogisticRegression
 predicitng topic severe_toxic
 predicitng topic identity_hate
 predicitng topic obscene
 predicitng topic insult
 predicitng topic threat
 predicitng topic toxic


In [17]:
# #Utest test predict
# #dfres = mySVC.predict(trn_term_doc, testid = train['id'])
# #display(dfres.head(10))
# #display(train.head(10)[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']])

# #Utest test save
# mdDB = pd.read_json("../modelDB/modelMetaDB.json")
# modelpath = 'cnn/' # NOTE: this is relative to the modelDB path
# print("current modelDB")
# display(mdDB)

# mdDB = mySVC.save(mdDB, 'utestModel', modelpath, modelType = 'testType')
# mdDB.to_json("../modelDB/modelMetaDB.json")

# #Utest test load
# loadedSVC = TrainedModelSVC().load( mdDB, 'utestModel', modelpath)
# for i in loadedSVC.md.keys():  
#     print( "{} - {}".format(i, type(loadedSVC.md[i]).__name__ ) )
# dfres_loaded = loadedSVC.predict(test_term_doc, testid = test['id'])
# assert(dfres_loaded.equals(dfres))

In [18]:
#generate submission file
dfres.to_csv('../submission/submission_'+modelNameKey+'.csv', index = False)

In [19]:
#model persist
mdDB = pd.read_pickle("../modelDB/modelMetaDB.pkl")
mdDB = mySVC.save(mdDB, modelNameKey, 'svc/', modelType = modelName)
#display(mdDB)
mdDB.to_pickle("../modelDB/modelMetaDB.pkl")

saving model
Info: custom saving options
modelType - SVC


date                                              model  \
0   1970-01-01 00:00:00  {"severe_toxic": ["cnn/CNN1d-Embedding_Random-...   
1   1970-01-01 00:00:00                                 cnn/utestModel.sav   
2            2018-02-13  {"severe_toxic": ["cnn/CNN1d-Embedding_Random-...   
3            2018-02-13                      svc/SVC_keywordTermFreq_1.sav   
4            2018-02-14  {"severe_toxic": ["cnn/CNN1d-Embedding_Random-...   
5            2018-02-14  {"MultiClassModel": ["cnn/CNN1d_MultiClass_Emb...   
6            2018-02-17  {"MultiClassModel": ["cnn/LSTMWithKeyWordTermF...   
7            2018-02-17  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
8            2018-02-17  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
9            2018-02-17  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
10           2018-02-17  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
11           2018-02-17  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
12           2018-02-17  {"MultiClassModel": ["cnn/LSTMWithKeyWordTermF...   
13           2018-02-18  {"MultiClassModel": ["cnn/LSTMWithtfIdf_MultiC...   
14           2018-02-18  {"MultiClassModel": ["cnn/LSTMWithtfIdf_MultiC...   
15           2018-02-19  {"MultiClassModel": ["cnn/LSTMWithtfIdf_MultiC...   
16           2018-02-19  {"MultiClassModel": ["cnn/LSTM_MultiClass_Embe...   
17           2018-02-19  {"MultiClassModel": ["cnn/LSTMWithtfIdf_MultiC...   
18           2018-02-19                                svc/SVC_tfidf_1.sav   
19           2018-02-19                  svc/SVC_tfidf_covCorrection_2.sav   
20           2018-02-19                          svc/SVC_tfidf_False_1.sav   
21           2018-02-19  {"MultiClassModel": ["cnn/LSTMWithtfIdf_MultiC...   
22           2018-02-19                           svc/SVC_tfidf_True_1.sav   
23           2018-02-20                           svc/SVC_tfidf_True_2.sav   
24           2018-02-20                 svc/SVC_tfidf+charNGram_True_1.sav   
25           2018-02-21                       svc/SVC_charNGram_True_1.sav   
26           2018-02-21                      svc/SVC_charNGram_False_1.sav   
27           2018-02-21                svc/SVC_charNGram+tfidf_False_1.sav   
28           2018-02-21                 svc/SVC_charNGram+tfidf_True_1.sav   
29           2018-02-21     svc/SVC_charNGram+tfidf_True_maxAbsScale_1.sav   
30           2018-02-21               svc/SVC_tfidf_True_maxAbsScale_1.sav   
31           2018-02-21  svc/SVC_tfidf+tfIdf_charNGram_True_maxAbsScale...   
32           2018-02-21  svc/SVC_tfidf+tfIdf_charNGram_False_maxAbsScal...   
33           2018-02-22  svc/SVC_tfidf+tfIdf_charNGram_False_noScale_1.sav   

                                            modelName  \
0                  CNN1d-Embedding_Random-OneperClass   
1                                          utestModel   
2            CNN1d-Embedding_Random-OneperClass_arch1   
3                               SVC_keywordTermFreq_1   
4            CNN1d-Embedding_Random-OneperClass_arch2   
5             CNN1d_MultiClass_Embedding_Random_arch1   
6   LSTMWithKeyWordTermFreq_MultiClass_Embedding_R...   
7              LSTM_MultiClass_Embedding_Random_arch1   
8              LSTM_MultiClass_Embedding_Random_arch2   
9              LSTM_MultiClass_Embedding_Random_arch3   
10             LSTM_MultiClass_Embedding_Random_arch4   
11             LSTM_MultiClass_Embedding_Random_arch5   
12  LSTMWithKeyWordTermFreq_MultiClass_Embedding_R...   
13    LSTMWithtfIdf_MultiClass_Embedding_Random_arch1   
14    LSTMWithtfIdf_MultiClass_Embedding_Random_arch2   
15    LSTMWithtfIdf_MultiClass_Embedding_Random_arch3   
16             LSTM_MultiClass_Embedding_Random_arch6   
17    LSTMWithtfIdf_MultiClass_Embedding_Random_arch4   
18                                        SVC_tfidf_1   
19                          SVC_tfidf_covCorrection_2   
20                                  SVC_tfidf_False_1   
21  LSTMWithtfIdf_MultiClass_Embed

In [20]:
## main findings
# tfIdf + ngram tfidf + svc gives best results thus far
#  scaling will hurt score
#  resample some tiems will hurt score (when feature space is small?), when feature dim is large, resample can help
#  conv correction thus far will hurt score

